In [1]:
import sys
from pathlib import Path

# add parent folder to the path
module_path = str(Path.cwd().parents[0])
if module_path not in sys.path:
    sys.path.append(module_path)

# PanRicci

In [2]:
import networkx as nx
import numpy as np
import math
import ot

## Load GFA

In [3]:
from panricci.utils.gfa_loader import GFALoader

In [4]:
gfa_loader = GFALoader(undirected=True)
nodes, edges, G = gfa_loader("../data/test1.gfa")

In [5]:
nodes

{'1': {'label': 'AC', 'len': 2, 'node_depth': 1.0},
 '2': {'label': 'AA', 'len': 2, 'node_depth': 0.5},
 '3': {'label': 'CCTAAA', 'len': 6, 'node_depth': 1.0}}

In [6]:
edges

[('1', '3'), ('1', '2'), ('2', '3')]

## Probability distribution for a node

In [7]:
from panricci.distributions.variation_graph import DistributionNodes
distribution_nodes = DistributionNodes(G, alpha=0.5)

In [8]:
distribution_nodes(node="2")

{'2': 0.5, '1': 0.25, '3': 0.25}

## Ricci Flow

In [12]:
from panricci.ricci_flow import RicciFlow
ricci_flow = RicciFlow(G, dist_nodes=distribution_nodes, dirsave_graphs="../output/test3/ricci-flow")

In [13]:
ricci_flow(iterations=10)

Ricci-Flow: 100%|██████████| 10/10 [00:00<00:00, 438.03it/s]


## Alignment of pangenome graphs